In [4]:
import sys
import json
from pprint import pprint
import csv
from elasticsearch import Elasticsearch
import pandas as pd
from elasticsearch.helpers import bulk

In [5]:
df = pd.read_csv('data.csv')

print(df.head())

     Name Family name  Age Honorifics
0     Ali      abbasi   16         Dr
1  hosein      Ahmadi   17         Mr


In [6]:
es = Elasticsearch(
    ['localhost'],
    port=9200
)

es.ping()

True

#### Not bulky

In [7]:
for _, row in df.iterrows():
    document = row.to_dict()
    es.index(index='persons', body=document, doc_type=csv)

#### Bulky

In [8]:
index = 'persons_bulk'

bulk_data = []
for _, row in df.iterrows():
    document = row.to_dict()
    action = {
        '_index': index,
        '_source': document
    }
    bulk_data.append(action)

bulk(es, bulk_data, index=index)

(2, [])

In [11]:
search_query = {
    "query": {
        "match_all": {}
    }
}

response = es.transport.perform_request(
    method='GET',
    url='/persons_bulk/_search',
    body=search_query
)

hits = response['hits']['hits']
for hit in hits:
    source = hit['_source']
    print(source)

{'Name': 'Ali', 'Family name': ' abbasi', 'Age': 16, 'Honorifics': ' Dr'}
{'Name': 'hosein', 'Family name': ' Ahmadi', 'Age': 17, 'Honorifics': ' Mr'}
{'Name': 'Ali', 'Family name': ' abbasi', 'Age': 16, 'Honorifics': ' Dr'}
{'Name': 'hosein', 'Family name': ' Ahmadi', 'Age': 17, 'Honorifics': ' Mr'}
{'Name': 'Ali', 'Family name': ' abbasi', 'Age': 16, 'Honorifics': ' Dr'}
{'Name': 'hosein', 'Family name': ' Ahmadi', 'Age': 17, 'Honorifics': ' Mr'}
